In [5]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from agent.model import TD3_Actor
from environment.env import TradingEnv

In [2]:
env = TradingEnv('data')

g:\Code\Python\GitHub\Portfolio-Optimizer\environment\env.py:242: DeprecationWarning: This function is deprecated. Please call randint(59, 840 + 1) instead
  self.dates_num-self.steps-1)
g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\pykalman\utils.py:111: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args = inspect.getargspec(obj.__init__)[0]


In [10]:
env.observation_space.sample()['portfolio'].shape

(1, 8, 60)

In [2]:
device = torch.device('cuda')

In [6]:
conv_port = nn.Conv2d(1, 2, (1, 58))

In [8]:
test = torch.randn((1, 8, 60))

In [15]:
conv_port(test.view(-1, 1, 8, 60))[0][0]

tensor([[ 0.2798, -0.2002, -0.2865],
        [ 0.1899,  0.0393, -0.0735],
        [-0.2225,  0.4772, -0.6020],
        [-0.3159,  0.0966, -0.2440],
        [ 0.5641, -0.5795, -0.9585],
        [-0.3121,  0.2926, -0.3872],
        [-1.1763,  0.7062,  0.2931],
        [-0.7268,  0.1898,  0.0464]], grad_fn=<SelectBackward0>)

In [3]:
test_model = TD3_Actor(device=device, model_type='Close').to(device)

In [20]:
df = pd.DataFrame(np.array(test_model.conv_port.weight.data[0][0][0].detach().cpu())).T

In [22]:
df_1 = pd.DataFrame(np.array(test_model.conv_port.weight.data[1][0][0].detach().cpu())).T

In [25]:
df.to_csv('test_weight.csv', index=False)

In [26]:
df_1.to_csv('test_weight.csv', mode='a', index=False, header=False)

In [27]:
df_2 = pd.read_csv('test_weight.csv')

In [34]:
test_obs = torch.randn((1, 1, 8, 60), device=device)
test_action = torch.randn((1, 1, 8, 1), device=device)

In [44]:
test = torch.randn((1, 21, 8, 1))

In [56]:
test.shape[1]

21

In [59]:
os.path.isdir('output_records')

True

In [54]:
pd.DataFrame(np.array(test[0][0].squeeze().detach().cpu())).T

,0,1,2,3,4,5,6,7
0,0.92005,1.305477,-1.021071,-1.149334,-1.186895,1.442417,-2.102093,1.592534
